# Sudoku Search for Training Data

In [34]:
import os
import uuid
import hashlib
import torch
import ray
from fastbook import *

In [35]:
imageDir = "./train/images"

In [36]:
hashCache = []

if ray.is_initialized() == False:
    ray.init()

In [37]:
urls = search_images_ddg('sudoku', max_images=300)
len(urls),urls[0]

(300,
 'https://kinderbilder.download/wp-content/uploads/2020/06/kombinatorik-ein-sudoku-benotigt-mindestens-17-vorgaben-bestimmt-fur-sudoku-kostenlos-drucken-schwer.jpg')

In [38]:
def calculateMd5(filename):
    runningMd5 = hashlib.md5()
    with open(filename, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            runningMd5.update(chunk)
    return runningMd5.hexdigest()

In [39]:
def doesContentAlreadyExist(filename, dir):
    newFileHash = calculateMd5(filename)

    if (len(hashCache) == 0):
        for f in os.listdir(dir):
            absFile = os.path.join(dir, f)
            if os.path.isfile(absFile):
                hash = calculateMd5(absFile)
                hashCache.append(hash)

    for hash in hashCache:
        if (hash == newFileHash):
            return True
            
    return False

In [40]:
for url in urls:
    url = url.split('?')[0]
    print ("Processing URL: " + url)
    e = re.findall('\.[a-z]+$', url)
    if (len(e)) == 1:
        fileExtension = e[0]
        randomName = str(uuid.uuid4().hex)
        filename = randomName + fileExtension
        tempFilename = "/tmp/" + filename
        
        download_url(url, tempFilename)

        if doesContentAlreadyExist(tempFilename, imageDir) == True:
            print ("Found: " + tempFilename)
            os.remove(tempFilename)
        else:
            print ("NEW: " + tempFilename)
            os.replace(tempFilename, imageDir + "/" + filename)

    else:
        print ("Unable to process image due to unexpected extension: " + url)

Processing URL: https://kinderbilder.download/wp-content/uploads/2020/06/kombinatorik-ein-sudoku-benotigt-mindestens-17-vorgaben-bestimmt-fur-sudoku-kostenlos-drucken-schwer.jpg


Found: /tmp/9faefb02ad3344628bae52cc17f156f9.jpg
Processing URL: https://creagame.com/shop/613/sudoku-9x9-niveau-facile-pack-n-1-de-10-grilles.jpg


Found: /tmp/1352140a86564c16b73151ae3ccb8d88.jpg
Processing URL: http://observer.case.edu/wp-content/uploads/2013/08/Sudoku-Answers-12-941x1024.jpg


Found: /tmp/25b25ab7da6646d1ba28332e9936fb58.jpg
Processing URL: http://3.bp.blogspot.com/_Kh0CZuWd0T8/Sn_2OoqPKwI/AAAAAAAAFEg/z_vhmFkfOws/s400/PRINTABLE+SUDOKU+PUZZLES+(3).gif


Found: /tmp/c783a2b6749f4ddfb9120bf2e3f1516d.gif
Processing URL: https://printable-crosswordpuzzles.com/wp-content/uploads/2019/06/sudoku-for-all-ages-plus-lots-of-other-printable-activities-for-kids-sudoku-puzzles-printable-6x6.png


...

NEW: /tmp/2cd606aceef14a1cbb38e992e65aa616.jpg
Processing URL: https://kinderbilder.download/wp-content/uploads/2020/06/sudoku-vorlagen-fur-kinder-6x6-kostenlos-herunterladen-und-ganzes-sudoku-zum-ausdrucken-sehr-schwer-scaled.jpg


Found: /tmp/ad41459feb914287b19e516f1b20cf88.jpg
Processing URL: http://sudoku-instructions.com/EnglishSudokuCourse/wpimages/Sudoku%20Instructions%20Program%20-%20printing%20of%20sudoku%20puzzles.jpg


Found: /tmp/e99a77d108f947cb8d1a91e6a8a313c3.jpg
Processing URL: https://images0.persgroep.net/rcs/8tBMR78sgCPuq_wZQBgKvLeRNNg/diocontent/169235734/_fill/900/900/
Unable to process image due to unexpected extension: https://images0.persgroep.net/rcs/8tBMR78sgCPuq_wZQBgKvLeRNNg/diocontent/169235734/_fill/900/900/
Processing URL: https://i.pinimg.com/originals/ce/48/56/ce4856cd7ec5e4aaf10ac3985c5a4067.jpg


Found: /tmp/bbf10fcacbab44e79af0e131ffd0f46e.jpg
Processing URL: http://www.gamesavenue.net/gallery/mathematics-puzzle-games/sudoku_puzzle_game-copy.jpg


URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [ ]:
ds = ray.data.read_images(imageDir)
ds.size_bytes()
